In [ ]:
import discord
from discord.ext import tasks, commands
from datetime import datetime
import requests
import bs4
import nest_asyncio
nest_asyncio.apply()

file = open("token.txt", mode="r")
token = file.readline().strip()
general_channel = int(file.readline().strip())

In [ ]:
class Coin:
    def __init__(self, url, symbol):
        self.symbol = symbol
        self.price = self.coin_price(url)
        self.url = url
        
    def coin_price(self, url):
        try:
            req = requests.get(url)
            soup = bs4.BeautifulSoup(req.text)
            e = str(soup.find_all(attrs={'class': 'price'})[0])
            price = float(e[e.index("-->")+3:e.index("</")])
            return price
        except:
            return 0.01

    def print_price(self):
        st = ""
        current_price = self.coin_price(self.url)
        
        st += self.symbol + ": \n$" + str(current_price) + " (" + str(round(100*(current_price-self.price)/self.price, 2)) + "%) "
        st += ("🚀" if current_price > self.price else "📉" if current_price < self.price else "") + "\n\n"
        
        self.price = current_price
        return st

In [ ]:
client = commands.Bot(command_prefix = ".")
coins = []
coin_symbols = set()

@client.event
async def on_ready():
    channel = client.get_channel(general_channel)
    print_prices.start()
    print("Bot initialized")
    
    await channel.send("**Hello! Type !help for more information**")
    
@client.event
async def on_message(message):
    if "!" in message.content and message.content[0] == "!":
        if message.content.lower().strip() == "!help":
            st = "**INSTRUCTIONS**\n```"
            st += "1. Go to https://www.livecoinwatch.com \n"
            st += "2. Search your desired coin\n"
            st += "3. Add coin via command: !<url> (replace <url> with the url of the coin page)\n\n"
            st += "OR...\nType !coins to view active tokens."
            st += "```"
            await message.channel.send(st)
        elif message.content.lower().strip() == "!coins":
            if coins:
                st = "```" + str(len(coins)) + " active coins:\n\n"
                
                for i in range(len(coins)):
                    coins[i].price = coins[i].coin_price(coins[i].url)
                
                for i,j in enumerate(coins):
                    st += str(i+1) + ". " + j.symbol + " ($" + str(j.price) + ")" + "\n"
                st += "```"
            else:
                st = "No active coins. Add one now!"
                
            await message.channel.send(st)
            
        else:
            try:
                url = message.content[1:]
                symbol = message.content[message.content.index("-")+1:].strip().upper()
                new_coin = Coin(url, symbol)
                st = new_coin.print_price()
                
                if symbol not in coin_symbols:
                    coin_symbols.add(symbol)
                    coins.append(new_coin)
                    await message.channel.send("```Successfully added coin: " + symbol + " ✔```")
                    
                else:
                    await message.channel.send("```" + symbol + " is already added!```")
                    
            except:
                await message.channel.send("```Unable to parse link: " + message.content[1:] + " ❌```")
    
@tasks.loop(seconds=600)
async def print_prices():
    channel = client.get_channel(general_channel)
    
    if coins:
        message = "**⌚ PRICE UPDATE: *" + datetime.now().strftime("%H:%M:%S") + " EST***\n```"
        for coin in coins:
            message += coin.print_price()

        message += "```"
        await channel.send(message)
        
client.run(token)